In [1]:
import json
import pandas as pd
import requests
url = "http://bikeshare.metro.net/stations/json/"
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}
response = requests.get(url, headers = headers)

In [5]:
data = (json.loads(response.text))["features"]
data_list = []
for row in data:
    data_list.append({"KioskId": row["properties"]["kioskId"],
                      "Latitude" : row["properties"]["latitude"],
                      "Longitude": row["properties"]["longitude"],
                      "KioskPublicStatus" : row["properties"]["kioskPublicStatus"],
                      "KioskStatus" : row["properties"]["kioskStatus"],
                      "BikesAvailable" : row["properties"]["bikesAvailable"],
                      "DocksAvailable" : row["properties"]["docksAvailable"],
                      "TotalDocks" : row["properties"]["totalDocks"],
                      "CloseTime" : row["properties"]["closeTime"],
                      "OpenTime" : row["properties"]["openTime"],
                      "TimeZone" : row["properties"]["timeZone"],
                      "Address" : row["properties"]["addressStreet"],
                      "City" : row["properties"]["addressCity"],
                      "State" : row["properties"]["addressState"],
                      "ZipCode" : row["properties"]["addressZipCode"]
                       })

In [6]:
df = pd.DataFrame(data_list)
df.head()

,Address,BikesAvailable,City,CloseTime,DocksAvailable,KioskId,KioskPublicStatus,KioskStatus,Latitude,Longitude,OpenTime,State,TimeZone,TotalDocks,ZipCode
0,723 Flower Street,9,Los Angeles,23:58:00,18,3005,Active,FullService,34.04850,-118.25854,00:02:00,CA,Pacific Standard Time,27,90017
1,729 S Olive Street,14,Los Angeles,23:58:00,10,3006,Active,FullService,34.04554,-118.25667,00:02:00,CA,Pacific Standard Time,24,90014
2,557 S 5th Street,6,Los Angeles,23:58:00,17,3007,Active,FullService,34.05048,-118.25459,00:02:00,CA,Pacific Standard Time,23,90071
3,865 S Figueroa Street,10,Los Angeles,23:58:00,5,3008,Active,FullService,34.04661,-118.26273,00:02:00,CA,Pacific Standard Time,15,90017
4,401 East 11th Street,16,Los Angeles,23:58:00,6,3010,Active,FullService,34.03705,-118.25487,00:02:00,CA,Pacific Standard Time,22,90015
